In [818]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss, f1_score
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt

In [819]:
df = pd.read_csv('Glass.csv')
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,building_windows_float_processed
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,headlamps
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,headlamps
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,headlamps
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,headlamps


In [820]:
X = df.drop('Type' , axis = 1)
le = LabelEncoder()
y = df['Type']

le = LabelEncoder()
le_y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, le_y, test_size = 0.3, 
                                                    random_state=24,stratify=le_y)

In [821]:
X.shape , y.shape

((214, 9), (214,))

In [822]:
ohe = OneHotEncoder(sparse_output=False)

y_trn_ohe = ohe.fit_transform(y_train.reshape(-1,1))
y_trn_ohe.shape

(149, 6)

In [823]:
scaler = MinMaxScaler()
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [824]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_trn_ohe)

In [825]:
print(X_torch.size())
print(y_torch.size())

torch.Size([149, 9])
torch.Size([149, 6])


In [826]:
type(X_torch)

torch.Tensor

In [827]:
torch.manual_seed(24)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1],out_features=7),
                      nn.Tanh(),
                      # nn.SELU(),
                     nn.Linear(in_features=7,out_features=6))

In [828]:
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 7]              70
              Tanh-2                 [-1, 1, 7]               0
            Linear-3                 [-1, 1, 6]              48
Total params: 118
Trainable params: 118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [829]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [830]:
y_torch = y_torch
print(Y_torch.shape)
print(y_pred.shape)

torch.Size([149, 1, 1, 6])
(65,)


In [831]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

y_wt_sum = model(X_torch_test.float())
softmax = nn.Softmax(dim=1)
y_pred_prob = softmax(y_wt_sum)
y_pred_prob[:3]

tensor([[0.1921, 0.2151, 0.2227, 0.1307, 0.1474, 0.0920],
        [0.1928, 0.2103, 0.2090, 0.1552, 0.1411, 0.0916],
        [0.1888, 0.2216, 0.2287, 0.1265, 0.1457, 0.0886]],
       grad_fn=<SliceBackward0>)

In [832]:
for epoch in np.arange(0,10000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%1000 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  1.7068089246749878
epoch:  1001  loss:  0.8668758869171143
epoch:  2001  loss:  0.7779150605201721
epoch:  3001  loss:  0.7187986969947815
epoch:  4001  loss:  0.6707485914230347
epoch:  5001  loss:  0.6261810660362244
epoch:  6001  loss:  0.5841760635375977
epoch:  7001  loss:  0.549299955368042
epoch:  8001  loss:  0.5212037563323975
epoch:  9001  loss:  0.4975205957889557


In [833]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [834]:
y_wt_sum = model(X_torch_test.float())
softmax = nn.Softmax(dim=1)
y_pred_prob = softmax(y_wt_sum)
y_pred_prob[:3]

tensor([[9.9089e-01, 5.9265e-03, 4.5942e-06, 5.4639e-09, 1.3044e-08, 3.1795e-03],
        [1.6341e-12, 2.8909e-04, 1.4998e-05, 9.9945e-01, 2.4369e-04, 1.3662e-10],
        [3.2156e-01, 3.7133e-01, 7.6720e-07, 9.3465e-05, 1.5821e-02, 2.9119e-01]],
       grad_fn=<SliceBackward0>)

In [835]:
y_pred_prob =y_pred_prob.detach().numpy()
y_pred_prob.shape

(65, 6)

In [836]:
y_pred = np.argmax(y_pred_prob,axis=1)
y_pred.shape

(65,)

In [837]:
print(accuracy_score(y_test,y_pred))

0.676923076923077


In [838]:
log_loss(y_test,y_pred_prob)

1.0048998539143443

In [839]:
# SGD - Accuracy_score -0.6307
# AdaDelta LogLoss - 1.0303536507659252
# AdaGrad Logloss - 1.0084792072377773
# Adam - LogLoss -1.2247577582613132
# AdamW - LogLoss -1.1424108135482751
# Adammax - LogLoss -1.275936343294368
#LBFGS - Error (closure missing)
# Rprop Log Loss = 1.3210665269476123